In [1]:
import cdsapi 
import pandas as pd 

In [2]:
df = pd.read_excel('./persenDI.xlsx')
df = df[1:].rename({'วันที่สำรวจ': 'date', 'พิกัด': 'x', 'Unnamed: 6': 'y'}, axis=1).drop(columns='ลำดับ')
df

,date,ตำบล,อำเภอ,จังหวัด,x,y,อายุต้นยาง(ปี),ชนิดดิน,พันธุ์ยางพารา,โรคที่พบ,เปอร์เซนต์การเกิดโรค
1,2022-11-25,คองแห,หาดใหญ่,สงขลา,664830,780162,12,อื่นๆ,RRIM600,โรใบร่วงชนิดใหม่,25-50%
2,2022-11-25,ทุ่งตำเสา,หาดใหญ่,สงขลา,650795,771312,7,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,51-75%
3,2022-11-25,กำแพงเพรช,รัตภูมิ,สงขลา,638246,781871,13,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,25-50%
4,2022-11-25,เขาพระ,รัตภูมิ,สงขลา,633845,780484,12,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,25-50%
5,2022-11-25,ท่าชะม่วง,รัตภูมิ,สงขลา,631559,778930,12,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,11-25%
6,2022-11-26,เทพา,เทพา,สงขลา,711261,751139,2024-08-07 00:00:00,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,51-75%
7,2022-11-26,เทพา,เทพา,สงขลา,711914,754815,12,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,51-75%
8,2022-11-26,เกาะสะบ้า,เทพา,สงขลา,706333,758189,12,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,25-50%
9,2023-11-03,ท่าข้าม,หาดใหญ่,สงขลา,675181,778301,15,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,11-25%
10,2023-11-03,ท่าข้าม,หาดใหญ่,สงขลา,676166,778241,10,ร่วนบนทราย,RRIM600,โรใบร่วงชนิดใหม่,11-25%


In [3]:
import utm
import numpy as np
def utmToLatLon(x:list, y:list, zone:int = 47) -> tuple:
    x, y = np.mean(x), np.mean(y) #find mid point
    return utm.to_latlon(x,y, zone_number=zone, zone_letter='N') #return (lat, lon)

In [4]:
import sys
import os
sys.path.append('../')
sys.path.append('./WeatherApi')

DIR_OUTPUT = './process/1nc/'
os.makedirs(DIR_OUTPUT, exist_ok=True)
tumbonDic = []

c = cdsapi.Client()

for i, data in df.iterrows():

    if data['ตำบล'] not in tumbonDic:
        tumbonDic.append(data['ตำบล'])

        lat, lon = utmToLatLon(x=[data['x']], y=[data['y']], zone=47)
        date = str(data['date'].date()).split('-')
        year = date[0]
        month = int(date[1])

        print(f"\nDownloading {data['date'].date()}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']} ({lat}, {lon})")

        c.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'format': 'netcdf',
                    'variable': [
                        '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                        '2m_temperature', 'downward_uv_radiation_at_the_surface', 'evaporation',
                        'soil_temperature_level_1', 'surface_pressure', 'total_precipitation',
                        'uv_visible_albedo_for_diffuse_radiation', 'uv_visible_albedo_for_direct_radiation'
                    ],
                    'year' : year,
                    'month' : [month - 1, month],
                    'day' : [str(i) for i in range(1, 32)],
                    'time' : [
                        '00:00', '06:00', '12:00', '18:00'
                    ],
                    'area' : [lat, lon, lat, lon]
                },
                f"{DIR_OUTPUT}{i}-{data['date'].date()}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']}.nc")

2024-05-22 16:48:19,771 INFO Welcome to the CDS
2024-05-22 16:48:19,772 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:48:20,408 INFO Request is completed
2024-05-22 16:48:20,409 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data0/adaptor.mars.internal-1716115941.4652827-611-20-b2dacdad-c47e-4fd2-a9b2-68ee8073bb65.nc to ./process/1nc/1-2022-11-25-คองแห-หาดใหญ่-สงขลา.nc (9.9K)
2024-05-22 16:48:24,785 INFO Download rate 2.3K/s


2024-05-22 16:48:25,594 INFO Welcome to the CDS
2024-05-22 16:48:25,595 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:48:26,129 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data6/adaptor.mars.internal-1716117524.394392-7024-20-ff035a05-235f-4569-acc3-f6c46e41cf53.nc to ./process/1nc/2-2022-11-25-ทุ่งตำเสา-หาดใหญ่-สงขลา.nc (9.9K)
2024-05-22 16:48:29,935 INFO Download rate 2.6K/s


2024-05-22 16:48:30,808 INFO Welcome to the CDS
2024-05-22 16:48:30,809 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:48:31,368 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data9/adaptor.mars.internal-1716117607.4884462-12880-20-b52e13bc-ac21-4f7c-b726-346d603a1d21.nc to ./process/1nc/3-2022-11-25-กำแพงเพรช-รัตภูมิ-สงขลา.nc (9.9K)
2024-05-22 16:48:35,370 INFO Download rate 2.5K/s


2024-05-22 16:48:36,632 INFO Welcome to the CDS
2024-05-22 16:48:36,633 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:48:37,564 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data7/adaptor.mars.internal-1716117699.922205-31076-1-a70c4fa2-5868-49dd-a036-3b36d347b05e.nc to ./process/1nc/4-2022-11-25-เขาพระ-รัตภูมิ-สงขลา.nc (9.9K)
2024-05-22 16:48:51,071 INFO Download rate 748.4/s


2024-05-22 16:48:52,429 INFO Welcome to the CDS
2024-05-22 16:48:52,430 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:48:54,194 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data2/adaptor.mars.internal-1716117784.9371996-22112-17-75f1422f-4316-48ba-9c84-d74a576ac28d.nc to ./process/1nc/5-2022-11-25-ท่าชะม่วง-รัตภูมิ-สงขลา.nc (9.9K)
2024-05-22 16:49:01,329 INFO Download rate 1.4K/s


2024-05-22 16:49:02,758 INFO Welcome to the CDS
2024-05-22 16:49:02,759 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:49:03,476 INFO Request is queued
2024-05-22 16:49:05,316 INFO Request is running
2024-05-22 16:52:05,141 INFO Request is completed
2024-05-22 16:52:05,142 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data0/adaptor.mars.internal-1716371464.5958788-32026-7-a7dcd14d-e1e4-4714-92ab-69a529c35eec.nc to ./process/1nc/6-2022-11-26-เทพา-เทพา-สงขลา.nc (9.9K)
2024-05-22 16:52:09,346 INFO Download rate 2.3K/s


2024-05-22 16:52:10,151 INFO Welcome to the CDS
2024-05-22 16:52:10,152 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-22 16:52:10,615 INFO Request is queued
2024-05-22 16:52:12,837 INFO Request is running
